---

## ⚗️ LCEL 핵심 개념 이해하기

### 🔗 파이프라인(Pipeline)이란?

**LCEL의 핵심은 파이프라인 연산자 `|`** 입니다. 이는 **Unix 파이프라인**에서 영감을 받은 개념으로, 하나의 출력이 다음 단계의 입력으로 자연스럽게 전달됩니다.

#### 🏭 **공장 생산라인으로 이해하기**

```
📦 원재료 → 🔨 가공1 → ⚙️ 가공2 → 📋 검수 → ✅ 완제품
```

LCEL도 똑같은 방식으로 동작합니다:

```python
# 🔗 LCEL 체인 구성
chain = prompt_template | model | output_parser
#      ↑              ↑     ↑
#   1단계: 프롬프트 생성 → 2단계: AI 처리 → 3단계: 결과 파싱
```

### 🎯 기본 체인의 3단계 구조

#### 1️⃣ **PromptTemplate** - 📝 지시사항 준비
- **역할**: 사용자 입력을 AI가 이해할 수 있는 형태로 변환
- **입력**: 딕셔너리 형태의 변수들 (`{"topic": "인공지능"}`)
- **출력**: 완성된 프롬프트 문자열

#### 2️⃣ **Model** - 🤖 AI 처리 
- **역할**: 프롬프트를 받아 AI가 답변 생성
- **입력**: 포맷팅된 프롬프트 텍스트
- **출력**: AIMessage 객체 (내용 + 메타데이터)

#### 3️⃣ **OutputParser** - 🎁 결과 정리
- **역할**: AI 응답을 사용하기 쉬운 형태로 변환  
- **입력**: AIMessage 객체
- **출력**: 순수 텍스트 또는 구조화된 데이터

### 💡 파이프라인의 강력함

```python
# 각 단계를 개별적으로 실행하는 전통적 방식 ❌
formatted_prompt = prompt_template.format(country="대한민국")
ai_response = model.invoke(formatted_prompt)  
final_result = output_parser.parse(ai_response)

# LCEL로 한 번에 실행하는 방식 ✅
result = chain.invoke({"country": "대한민국"})
```

**한 줄로 전체 과정을 완료!** 이것이 LCEL의 매력입니다. 🚀

---

## 📝 PromptTemplate 완전 정복

**PromptTemplate**은 **동적 프롬프트**를 만들어주는 강력한 도구입니다. 마치 **편지 양식**에 이름만 바꿔서 여러 사람에게 보내는 것과 같습니다!

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain-Tutorial


### 🔧 PromptTemplate의 구성 요소

**PromptTemplate**은 **템플릿 엔진**의 역할을 하며, 사용자 입력을 받아 **동적으로 프롬프트를 생성**합니다.

#### 📋 **핵심 구성 요소**

1. **🎯 template**: 실제 프롬프트 내용이 담긴 문자열 템플릿
   ```python
   template = "{country}의 수도는 어디인가요?"
   ```

2. **🔑 input_variables**: 템플릿에서 사용할 변수들의 이름 목록
   ```python
   # 중괄호 {} 안의 변수명들이 input_variables가 됨
   # 위 예시에서는 ["country"]가 자동으로 추출됨
   ```

#### 💡 **템플릿 변수 사용법**

- **중괄호 `{}`** 안에 변수명 작성
- **여러 변수** 사용 가능: `"{name}님, {city}의 날씨는 어떤가요?"`
- **변수명은 영문자로 시작**, 숫자와 언더스코어 사용 가능

#### 🚀 **from_template() 메서드의 편리함**

```python
# ✅ 간단한 방법 - from_template() 사용 (권장)
prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")

# ❌ 복잡한 방법 - 직접 생성 (비권장)  
prompt = PromptTemplate(
    template="{country}의 수도는 어디인가요?",
    input_variables=["country"]
)
```

**from_template()의 장점:**
- **🔍 자동 변수 추출**: 중괄호 안의 변수를 자동으로 인식
- **✨ 간결한 코드**: 한 줄로 템플릿 생성 완료
- **🛡️ 오류 방지**: 변수명 오타나 누락 위험 최소화

실제로 PromptTemplate을 만들어보겠습니다! 💻

In [3]:
# 스트리밍 출력을 위한 헬퍼 함수와 PromptTemplate 클래스 임포트
from langchain_teddynote.messages import stream_response  
from langchain_core.prompts import PromptTemplate

### 🏗️ PromptTemplate 객체 생성하기

`from_template()` 메서드를 사용하면 간단하게 PromptTemplate 객체를 만들 수 있습니다.

In [4]:
# 동적 프롬프트 템플릿 정의 - {country} 부분이 변수로 대체됨
template = "{country}의 수도는 어디인가요?"

# from_template 메서드를 이용하여 PromptTemplate 객체 생성
# 중괄호 안의 변수들이 자동으로 input_variables로 인식됨
prompt_template = PromptTemplate.from_template(template)

# 생성된 PromptTemplate 객체 확인
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# 템플릿에 구체적인 값을 대입하여 완성된 프롬프트 생성
prompt = prompt_template.format(country="대한민국")
print(f"생성된 프롬프트: {prompt}")

생성된 프롬프트: 대한민국의 수도는 어디인가요?


In [6]:
# 다른 국가로 변경하여 프롬프트 생성 테스트
prompt = prompt_template.format(country="미국")
print(f"생성된 프롬프트: {prompt}")

생성된 프롬프트: 미국의 수도는 어디인가요?


In [7]:
# ChatOpenAI 모델 임포트 및 초기화
from langchain_openai import ChatOpenAI

# OpenAI GPT 모델 객체 생성
model = ChatOpenAI(
    model="gpt-4.1",  # 사용할 모델 지정
    temperature=0.1,  # 창의성 조절 (낮을수록 일관된 답변)
)

In [8]:
response = model.invoke(prompt)
print(response.content)

미국의 수도는 워싱턴 D.C.(Washington, D.C.)입니다.


---

## 🔗 LCEL 체인 생성 - 파이프라인의 마법

### 🎯 LCEL(LangChain Expression Language) 심화 이해

![LCEL Pipeline](./images/lcel.png)

**LCEL의 핵심은 `|` (파이프) 연산자**입니다. 이를 통해 여러 구성 요소를 **체인처럼 연결**하여 하나의 통합된 워크플로우를 만들 수 있습니다.

### 🔄 파이프라인 연산자의 작동 원리

```python
chain = prompt_template | model | output_parser
```

#### 📊 데이터 흐름 과정

1. **📥 입력**: `{"topic": "인공지능"}` (딕셔너리)
2. **📝 1단계**: `prompt_template` → 완성된 프롬프트 텍스트
3. **🤖 2단계**: `model` → AIMessage 객체 (AI 응답)  
4. **🎁 3단계**: `output_parser` → 최종 텍스트 결과

### 🛠️ Unix 파이프라인과의 유사점

**Unix 명령어**와 개념이 매우 유사합니다:

```bash
# Unix 파이프라인 예시
cat file.txt | grep "keyword" | wc -l
```

```python
# LCEL 파이프라인 예시  
chain = prompt | model | parser
```

**공통점:**
- **⬅️ 왼쪽에서 오른쪽**으로 데이터 흐름
- **🔗 각 단계의 출력**이 다음 단계의 입력이 됨
- **🧩 모듈화**: 각 구성요소를 독립적으로 테스트하고 교체 가능

### 💡 LCEL이 혁신적인 이유

#### ✨ **자동 최적화**
- **병렬 처리**: 가능한 부분은 동시에 실행
- **메모리 효율성**: 중간 결과의 스마트한 관리
- **스트리밍**: 실시간 결과 출력 지원

#### 🔧 **개발자 친화적**
- **직관적 문법**: 데이터 흐름을 한눈에 파악
- **디버깅 용이**: 각 단계별 결과 추적 가능
- **재사용성**: 구성요소를 다른 체인에서도 활용

이제 실제로 간단한 체인을 만들어봅시다! 🚀

In [9]:
# 교육용 프롬프트 템플릿 생성 - 복잡한 주제를 쉽게 설명하도록 지시
prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 설명해주세요.")

# ChatOpenAI 모델 객체 생성
model = ChatOpenAI(model="gpt-4.1", temperature=0.1)

# 기본 체인 구성 (출력 파서 없이) - 프롬프트와 모델만 연결
chain = prompt | model

### ⚡ invoke() 메서드 - 체인 실행하기

**invoke()** 는 LCEL 체인을 실행하는 가장 기본적인 메서드입니다.

#### 📋 **사용법**
- **입력 형태**: Python 딕셔너리 `{"변수명": "값"}`
- **실행 방식**: 동기식 (결과가 나올 때까지 대기)
- **반환값**: 체인의 최종 출력 (출력 파서에 따라 달라짐)

In [10]:
# 체인 실행을 위한 입력 딕셔너리 정의
# 키는 템플릿의 변수명과 일치해야 함
input = {"topic": "인공지능 모델의 학습 원리"}

In [11]:
# LCEL 체인 실행: 프롬프트 생성 → 모델 처리 → AIMessage 반환
# invoke() 메서드로 전체 파이프라인을 한 번에 실행
result = chain.invoke(input)

# 결과 출력 (AIMessage 객체 형태로 반환됨)
result

AIMessage(content='네, 인공지능 모델의 학습 원리를 쉽게 설명해드릴게요!\n\n---\n\n### 1. **예시로 이해하기**\n인공지능 모델을 "학생"이라고 생각해보세요. 이 학생에게 "고양이와 개 사진을 구분하는 법"을 가르치고 싶다고 합시다.\n\n---\n\n### 2. **학습 데이터**\n먼저, 학생에게 많은 고양이 사진과 개 사진을 보여줍니다. 각 사진에는 "이건 고양이야", "이건 개야"라고 정답(라벨)이 붙어 있습니다.\n\n---\n\n### 3. **패턴 찾기**\n학생(모델)은 사진들을 보면서 고양이와 개의 차이점을 스스로 찾아내려고 노력합니다. 예를 들어, "귀가 뾰족하면 고양이일 확률이 높네?" 같은 식으로요.\n\n---\n\n### 4. **예측과 정답 비교**\n학생이 새 사진을 보고 "이건 고양이야!"라고 예측하면, 선생님(컴퓨터)이 "맞았어!" 또는 "틀렸어!"라고 알려줍니다.\n\n---\n\n### 5. **틀린 부분 수정**\n틀렸을 때는 "왜 틀렸지?"를 생각하고, 다음에는 더 잘 맞추려고 머릿속 기준(모델의 내부 구조)을 조금씩 바꿉니다. 이 과정을 반복하면서 점점 더 잘 맞추게 됩니다.\n\n---\n\n### 6. **반복 학습**\n이렇게 예측 → 정답 확인 → 수정 과정을 수천, 수만 번 반복하면서, 학생(모델)은 고양이와 개를 구분하는 법을 점점 더 잘 배우게 됩니다.\n\n---\n\n### 7. **정리**\n즉, 인공지능 모델의 학습 원리는 **많은 예시(데이터)를 보고, 예측을 해보고, 정답과 비교해서, 틀린 부분을 고쳐가며 점점 더 똑똑해지는 과정**입니다.\n\n---\n\n궁금한 점이 더 있으시면 언제든 질문해주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 452, 'prompt_tokens': 21, 'total_tokens': 473, 'comple

### 🌊 스트리밍 출력의 강력함

**스트리밍**은 AI가 응답을 생성하는 **실시간 과정을 관찰**할 수 있게 해주는 기능입니다. 마치 사람이 말하는 것처럼 단어 하나씩 나타납니다!

In [12]:
# 스트리밍 방식으로 체인 실행 - 실시간으로 응답 생성 과정 확인
answer = chain.stream(input)

# langchain_teddynote의 헬퍼 함수로 스트리밍 출력을 깔끔하게 표시
stream_response(answer)

물론입니다! 인공지능 모델, 특히 많이 쓰이는 **딥러닝** 모델의 학습 원리를 쉽게 설명해드릴게요.

---

### 1. **모델은 뇌의 신경망을 흉내냅니다**
- 인공지능 모델은 사람의 뇌처럼 **입력(정보)**을 받아 **출력(결과)**을 내는 구조입니다.
- 이 과정에서 **뉴런(노드)**과 **가중치(연결 강도)**를 사용해 정보를 처리합니다.

---

### 2. **학습이란?**
- **학습**이란, 모델이 **정답을 맞히는 방법을 스스로 찾아가는 과정**입니다.
- 예를 들어, 고양이와 강아지 사진을 구분하는 모델을 만든다고 해볼게요.

---

### 3. **학습 과정 단계별 설명**

#### (1) **입력과 정답 준비**
- 고양이 사진 → "고양이" (정답)
- 강아지 사진 → "강아지" (정답)

#### (2) **예측**
- 모델이 사진을 보고 "고양이" 또는 "강아지"라고 예측합니다.

#### (3) **오답 확인**
- 예측이 정답과 다르면, **얼마나 틀렸는지(오차)**를 계산합니다.

#### (4) **가중치 조정**
- 오차가 작아지도록 모델 내부의 **가중치**를 조금씩 바꿉니다.
- 이 과정을 **수천~수백만 번 반복**합니다.

#### (5) **점점 똑똑해짐**
- 반복할수록 모델은 정답을 더 잘 맞히게 됩니다.

---

### 4. **비유로 이해하기**
- **처음 자전거를 배우는 것**과 비슷합니다.
  - 처음엔 자주 넘어지지만, 넘어질 때마다 몸의 균형을 조금씩 조정하죠.
  - 계속 연습하면 점점 잘 타게 됩니다.
- 인공지능 모델도 **실수(오차)를 줄이면서** 점점 더 잘 맞히게 됩니다.

---

### 5. **정리**
- 인공지능 모델의 학습은 **입력과 정답을 많이 보여주면서**, **오답을 줄이도록 내부 구조(가중치)를 조정하는 과정**입니다.
- 이 과정을 반복하면, 모델은 새로운 입력도 잘 맞히게 됩니다.

---

궁금한 점이 더 있으면 언제든 질문해주세요!

---

## 🎁 OutputParser - 결과를 원하는 형태로

**OutputParser**는 AI의 복잡한 응답을 **사용하기 쉬운 형태로 변환**해주는 마지막 단계입니다.

In [13]:
# 문자열 출력 파서 임포트
from langchain_core.output_parsers import StrOutputParser

# StrOutputParser 객체 생성 - AIMessage에서 순수 텍스트만 추출
output_parser = StrOutputParser()

### 🔗 완전한 체인 구성하기

이제 **3단계 파이프라인**을 완성해봅시다: **PromptTemplate → Model → OutputParser**

In [14]:
# 완전한 LCEL 체인 구성: 프롬프트 → 모델 → 출력 파서
# 이제 결과가 AIMessage가 아닌 순수 문자열로 반환됨
chain = prompt | model | output_parser

In [15]:
# 완성된 체인으로 invoke 실행 - 이제 순수 문자열이 반환됨
input = {"topic": "인공지능 모델의 학습 원리"}
result = chain.invoke(input)

# 결과 출력 (이제 문자열 형태로 깔끔하게 출력됨)
print("=== 완성된 체인 결과 ===")
print(result)

=== 완성된 체인 결과 ===
네, 인공지능 모델의 학습 원리를 쉽게 설명해드릴게요!

---

### 1. **예시로 이해하기**
인공지능 모델을 **학생**이라고 생각해보세요. 이 학생에게 **문제(입력)**와 **정답(출력)**을 많이 보여주면서 공부를 시키는 과정이 바로 **학습**입니다.

---

### 2. **학습의 과정**
1. **데이터 준비**
   - 학생에게 수많은 문제와 정답을 준비해줍니다. (예: 고양이 사진 → "고양이", 강아지 사진 → "강아지")

2. **예측하기**
   - 학생이 문제를 보고 답을 맞혀봅니다. (예: 사진을 보고 "고양이"라고 대답)

3. **정답과 비교**
   - 학생의 답이 맞았는지 틀렸는지 확인합니다.

4. **틀린 부분 고치기**
   - 틀렸다면, 왜 틀렸는지 알려주고, 다음에는 더 잘 맞힐 수 있도록 방법을 조금씩 바꿉니다. (이 과정을 **모델의 파라미터를 조정한다**고 해요)

5. **반복**
   - 이 과정을 수천, 수만 번 반복하면서 학생(모델)은 점점 더 똑똑해집니다.

---

### 3. **핵심 원리**
- **경험을 통해 배우기**: 많은 예시(데이터)를 통해 규칙을 스스로 찾아냅니다.
- **오답 수정**: 틀린 답을 할 때마다 조금씩 방법을 바꿔가며 더 나은 답을 하도록 만듭니다.
- **반복 학습**: 여러 번 반복해서 점점 더 정확해집니다.

---

### 4. **비유로 정리**
- 인공지능 모델은 **학생**,
- 데이터는 **문제집**,
- 학습은 **문제 풀고 오답노트 정리하는 과정**입니다.

---

궁금한 점이 더 있으면 언제든 질문해주세요!


In [16]:
# 완성된 체인으로 스트리밍 실행
answer = chain.stream(input)

print("=== 스트리밍 출력 ===")
# 실시간으로 문자열이 생성되는 과정을 관찰
stream_response(answer)

=== 스트리밍 출력 ===
네, 인공지능 모델의 학습 원리를 쉽게 설명해드릴게요.

---

### 1. **예시로 이해하기**
인공지능 모델을 마치 어린아이가 사과와 바나나를 구분하는 법을 배우는 것에 비유할 수 있어요.

- **데이터 제공:** 아이에게 사과와 바나나 사진을 많이 보여주면서 "이건 사과야", "이건 바나나야"라고 알려줍니다.
- **패턴 찾기:** 아이는 여러 사진을 보면서 사과는 둥글고 빨갛고, 바나나는 길쭉하고 노랗다는 특징을 스스로 발견합니다.
- **새로운 사진 구분:** 이제 아이에게 처음 보는 과일 사진을 보여주면, 배운 특징을 바탕으로 "이건 사과야!"라고 맞힐 수 있게 됩니다.

---

### 2. **인공지능 모델의 실제 학습 과정**
- **데이터 입력:** 컴퓨터에게 많은 데이터를 줍니다. 예를 들어, 고양이와 강아지 사진과 각각의 정답(라벨)을 함께 줍니다.
- **모델이 예측:** 컴퓨터(모델)가 사진을 보고 "이건 고양이야" 또는 "이건 강아지야"라고 예측합니다.
- **정답과 비교:** 모델의 예측이 실제 정답과 얼마나 다른지 비교합니다.
- **오류 수정:** 틀린 부분을 조금씩 고치면서(수학적으로는 '가중치'를 조정) 점점 더 잘 맞히도록 학습합니다.
- **반복:** 이 과정을 수천~수만 번 반복하면서 점점 더 똑똑해집니다.

---

### 3. **핵심 요약**
- **많은 예시(데이터)를 보고**
- **특징(패턴)을 스스로 찾고**
- **틀린 부분을 고치면서**
- **점점 더 정확하게 예측하는 법을 배우는 것**이 인공지능 모델의 학습 원리입니다.

---

궁금한 점이 더 있으시면 언제든 질문해주세요!

---

## 🎯 실전 프로젝트: 영어 회화 튜터 AI 만들기

이제 배운 내용을 활용해서 **실용적인 영어 학습 도우미**를 만들어봅시다! 

### 💡 프로젝트 개요

- **목표**: 상황별 영어 회화 생성 + 한글 번역 제공
- **특징**: 체계적인 포맷으로 학습 효과 극대화
- **활용**: 다양한 상황에 맞는 영어 표현 학습

In [17]:
# 전문적인 영어 회화 튜터 프롬프트 템플릿 설계
template = """You are an experienced English conversation teacher with 10 years of expertise.
Create practical English conversations for the given situation with Korean translations.
Please follow the FORMAT exactly as shown below.

#SITUATION:
{question}

#FORMAT:
- English Conversation:
- Korean Translation:
- Useful Expressions:
- Cultural Notes (if applicable):
"""

# 개선된 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 모델 객체 생성 (최신 모델 사용)
model = ChatOpenAI(model_name="gpt-4.1")

# 문자열 출력 파서 생성
output_parser = StrOutputParser()

In [18]:
# 영어 회화 튜터 체인 구성
# 프롬프트 → 모델 → 출력 파서의 완전한 파이프라인
chain = prompt | model | output_parser

In [19]:
# 첫 번째 상황: 식당에서 음식 주문하기
situation_1 = "저는 식당에 가서 음식을 주문하고 싶어요"

print("🍽️ === 식당 주문 상황 ===")
print(chain.invoke({"question": situation_1}))

🍽️ === 식당 주문 상황 ===
#SITUATION:  
저는 식당에 가서 음식을 주문하고 싶어요

- English Conversation:  
Waiter: Hello, welcome! How many people are in your party?  
You: Just one, please.  
Waiter: This way, please. Here is the menu.  
You: Thank you.  
Waiter: Are you ready to order?  
You: Yes. I’d like the chicken salad, please.  
Waiter: Would you like something to drink?  
You: Just water, please.  
Waiter: Sure. Your order will be ready soon.  

- Korean Translation:  
점원: 안녕하세요, 어서 오세요! 몇 분이세요?  
당신: 한 명입니다.  
점원: 이쪽으로 오세요. 여기 메뉴판입니다.  
당신: 감사합니다.  
점원: 주문하시겠어요?  
당신: 네. 치킨 샐러드 주세요.  
점원: 음료는 무엇으로 드릴까요?  
당신: 물만 주세요.  
점원: 네, 곧 준비해 드릴게요.  

- Useful Expressions:  
- I’d like to order. (주문하고 싶어요.)  
- Can I have the menu, please? (메뉴판 좀 주시겠어요?)  
- What do you recommend? (추천해주실 만한 게 있나요?)  
- I’ll have this, please. (이거로 할게요.)  
- Can I get it to go? (포장해 주실 수 있나요?)  
- Could I get the bill, please? (계산서 주세요.)  

- Cultural Notes (if applicable):  
In many Western restaurants, you will be seated by 

In [20]:
# 두 번째 상황: 스트리밍으로 실시간 학습 경험
situation_2 = "미국에서 피자 주문"

print("🍕 === 미국 피자 주문 상황 (스트리밍) ===")
# 스트리밍으로 영어 회화가 실시간으로 생성되는 과정 관찰
answer = chain.stream({"question": situation_2})
stream_response(answer)

🍕 === 미국 피자 주문 상황 (스트리밍) ===
#SITUATION:  
미국에서 피자 주문

- English Conversation:  
Staff: Hi, thank you for calling Tony's Pizza. How can I help you?  
Customer: Hi, I'd like to order a large pepperoni pizza for delivery, please.  
Staff: Sure! Would you like any extra toppings?  
Customer: Yes, can I add mushrooms and olives?  
Staff: Of course. Anything else?  
Customer: That’s it. How long will it take?  
Staff: About 30 minutes. Can I get your address, please?  
Customer: Yes, it’s 123 Maple Street, Apartment 4B.  
Staff: Got it. Your total is $18.50. Would you like to pay with cash or card?  
Customer: I’ll pay with my card.  
Staff: Perfect. I’ll take your payment details now. Thank you for your order!

- Korean Translation:  
직원: 안녕하세요, 토니 피자입니다. 무엇을 도와드릴까요?  
손님: 안녕하세요. 배달로 큰 페퍼로니 피자 한 판 주문하고 싶어요.  
직원: 네! 추가 토핑 원하시는 게 있으신가요?  
손님: 네, 버섯이랑 올리브 추가할 수 있을까요?  
직원: 물론이죠. 다른 건 더 필요하신가요?  
손님: 아니요, 그게 다예요. 얼마나 걸리죠?  
직원: 약 30분 정도 걸릴 것입니다. 주소를 알려주시겠어요?  
손님: 네, 메이플 스트리트 123번지, 아파트 4B입니다

In [21]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

#SITUATION:
저는 식당에 가서 음식을 주문하고 싶어요

- English Conversation:  
A: Hello, welcome to our restaurant. How many people are in your party?  
B: Hi, just one person.  
A: Right this way, please. Here is the menu.  
B: Thank you.  
A: Are you ready to order?  
B: Yes, I’d like to have the chicken pasta, please.  
A: Would you like anything to drink?  
B: Just water, please.  
A: Sure. Your order will be ready soon.

- Korean Translation:  
A: 안녕하세요, 저희 레스토랑에 오신 것을 환영합니다. 몇 분이세요?  
B: 안녕하세요, 저 혼자예요.  
A: 이쪽으로 오세요. 여기 메뉴입니다.  
B: 감사합니다.  
A: 주문하시겠어요?  
B: 네, 치킨 파스타 하나 주세요.  
A: 음료는 무엇으로 드릴까요?  
B: 물만 주세요.  
A: 알겠습니다. 곧 준비해드리겠습니다.

- Useful Expressions:  
- How many people? (몇 분이세요?)  
- Here is the menu. (여기 메뉴입니다.)  
- Are you ready to order? (주문하시겠어요?)  
- I’d like to have ____, please. (__ 주세요.)  
- Would you like anything to drink? (음료는 무엇으로 드릴까요?)  
- Just water, please. (물만 주세요.)  
- Your order will be ready soon. (곧 준비해드리겠습니다.)

- Cultural Notes (if applicable):  
In many Western restaur

In [22]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

#SITUATION:  
미국에서 피자 주문

- English Conversation:  
Clerk: Hello, thank you for calling Tony’s Pizza. How can I help you?  
Customer: Hi, I’d like to order a pizza for delivery, please.  
Clerk: Sure! What size pizza would you like?  
Customer: Large, please.  
Clerk: And what toppings would you like?  
Customer: Pepperoni and mushrooms.  
Clerk: Got it. That’s a large pizza with pepperoni and mushrooms. Can I get your address, please?  
Customer: Sure, it’s 123 Oak Street, Apartment 5.  
Clerk: And your phone number?  
Customer: 555-1234.  
Clerk: Great. Your total will be $18.50 and it should arrive in about 30 minutes.  
Customer: Thank you!  
Clerk: You’re welcome. Enjoy your pizza!

- Korean Translation:  
점원: 안녕하세요, Tony’s Pizza입니다. 무엇을 도와드릴까요?  
손님: 안녕하세요. 배달로 피자 한 판 주문하고 싶어요.  
점원: 네! 어떤 사이즈로 드릴까요?  
손님: Large(라지)로 주세요.  
점원: 토핑은 무엇으로 하시겠어요?  
손님: 페퍼로니하고 버섯이요.  
점원: 확인했습니다. 라지 사이즈 페퍼로니, 버섯 피자 맞으시죠? 주소 알려주시겠어요?  
손님: 네, 오크 스트리트 123번지, 아파트 5호입니다.  
점원: 전화번호도 부탁드립니다.  
손님: 555-123